## Environment Setup

In [ ]:
!pip install tensorboardX
!pip install pyglet==1.5.1
!pip install torchsummary
!pip install optuna
!pip install optuna-dashboard

In [1]:
!pip install torchrl
!pip install setuptools==65.5.1
!pip install gym==0.21.0
!pip install stable-baselines3[extra]
!pip install lz4

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 5.6 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.1/130.1 kB 10.9 MB/s eta 0:00:00
DEPRECATION: distro-info 0.23ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.36ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Defaulting to user installation because nor

In [ ]:
# !sudo apt-get install -y xvfb
!pip install pyvirtualdisplay

In [ ]:
!nvidia-smi

## Imports

In [3]:
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1024, 768))
virtual_display.start()

In [4]:
import sys
import os

# Get the absolute path to the parent directory of gym-tetris
gym_tetris_parent_path = os.path.abspath(os.path.join('..', 'gym-tetris'))

# Append the path to the sys.path
sys.path.append(gym_tetris_parent_path)

In [5]:
import random
import time
from distutils.util import strtobool

# import gym
from gym import Wrapper, ObservationWrapper
from gym.wrappers import RecordEpisodeStatistics, RecordVideo, FrameStack
from gym.spaces import Box, Discrete

import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from stable_baselines3.common.buffers import ReplayBuffer
from torch.utils.tensorboard import SummaryWriter
from torchrl.data import ListStorage, PrioritizedReplayBuffer

from nes_py.wrappers import JoypadSpace
import gym_tetris
from gym_tetris.actions import SIMPLE_MOVEMENT

from torchsummary import summary
from collections import deque

## Model

In [4]:
class QNetwork(nn.Module):
    def __init__(self, actions_num, frame_stack=1):
        super().__init__()
        self.network =  nn.Sequential(
            # (frame_stack, 20, 10)
            nn.Flatten(),
            # 200 x frame_stack
            nn.Linear(200*frame_stack, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, actions_num),
        )

    def forward(self, x):
        return self.network(x)


def linear_schedule(start_e: float, end_e: float, duration: int, t: int):
    slope = (end_e - start_e) / duration
    return max(slope * t + start_e, end_e)

In [ ]:
# Convolutional on 20x10
# class QNetwork(nn.Module):
#     def __init__(self, actions_num, frame_stack=1):
#         super().__init__()
#         self.network =  nn.Sequential(
#             # (frame_stack, 20, 10)
#             nn.Conv2d(1, 32, 4, stride=2),
#             nn.ReLU(),
#             # (32, 9, 4)
#             nn.Conv2d(32, 64, 2, stride=1),
#             nn.ReLU(),
#             # (32, 8, 3)
#             nn.Flatten(),
#             # 768
#             nn.Linear(768, 512),
#             nn.ReLU(),
#             nn.Linear(512, actions_num)
#         )

#     def forward(self, x):
#         return self.network(x)


# def linear_schedule(start_e: float, end_e: float, duration: int, t: int):
#     slope = (end_e - start_e) / duration
#     return max(slope * t + start_e, end_e)

## Training

In [5]:
class FrameSkipEnv(Wrapper):
    def __init__(self, env=None, skip=4):
        super(FrameSkipEnv, self).__init__(env)
        self._skip = skip

    def step(self, action):
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            # Only do the action on the first frame (action 0 is always NOOP)
            real_action = 0 if (i > 0) else action
            obs, reward, done, info = self.env.step(real_action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, info

    def reset(self):
        obs = self.env.reset()
        return obs

In [6]:
GAME_BOX = 47, 95, 209, 176
BOARD_SHAPE = 20, 10
y_step = (GAME_BOX[2] - GAME_BOX[0]) // BOARD_SHAPE[0]
x_step = (GAME_BOX[3] - GAME_BOX[1]) // BOARD_SHAPE[1]

# Given an image of the current board, obtain a binary (20x10) representation
class BinaryBoard(ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.observation_space = Box(0, 1, BOARD_SHAPE)

    def observation(self, obs):
        gray = np.mean(obs, axis=-1)
        cropped = gray[GAME_BOX[0]+(y_step//2) : GAME_BOX[2] : y_step,
                       GAME_BOX[1]+(x_step//2) : GAME_BOX[3] : x_step]
        assert cropped.shape == BOARD_SHAPE
        cropped[cropped > 1] = 1
        return cropped

In [7]:
INNER_SKIP = 16
OUTER_SKIP = 3
# Making an environment
def get_env(env_id, seed, capture_video, run_name, video_freq=100, frame_stack=4, num_reloads=0):
    env = gym_tetris.make(env_id)
    env = JoypadSpace(env, SIMPLE_MOVEMENT)
    
    env = RecordEpisodeStatistics(env)
    if capture_video:
        env = RecordVideo(env, f"videos/{run_name}_{num_reloads}", episode_trigger=lambda ep_num: ep_num % video_freq == 0)
    
    env = BinaryBoard(env)
    env = FrameSkipEnv(env, skip=INNER_SKIP)
    env = FrameStack(env, frame_stack)
    env = FrameSkipEnv(env, skip=OUTER_SKIP)

    env.seed(seed)
    env.action_space.seed(seed)
    env.observation_space.seed(seed)
    return env

In [8]:
FPS = 60 / (INNER_SKIP * OUTER_SKIP)
SCALE_UP = 10

In [9]:
# Evaluation
def evaluate(
    model: torch.nn.Module,
    env_id: str,
    eval_episodes: int,
    run_name: str,
    seed: int,
    device: torch.device = torch.device("cpu"),
    capture_video: bool = True,
    video_frequency: int = 1,
    frame_stack: int = 1
):
    env = get_env(env_id, seed, capture_video, run_name, video_frequency, frame_stack) 
    
    model.eval()

    scores = []
    for episode in range(eval_episodes):
        if capture_video:
            out = cv2.VideoWriter(f'eval_episode{episode}.mp4', cv2.VideoWriter_fourcc(*'mp4v'), FPS, (BOARD_SHAPE[1]*SCALE_UP, BOARD_SHAPE[0]*SCALE_UP), False)
        
        obs = env.reset()
        done = False
        while not done:
            if capture_video:
                img = np.array(obs).astype('uint8')
                if frame_stack > 1:
                    img = img[-1]
                out.write(img)

            input = np.expand_dims(np.array(obs), axis=0)
            q_values = model(torch.Tensor(input).to(device))
            action = int(torch.argmax(q_values))
            obs, _, done, info = env.step(action)
        
        print(f"eval_episode={len(scores)}, score={info.get('score')}, episodic_return={info.get('episode')['r']}")
        scores.append(info.get("score"))

    env.close()
    return scores

In [17]:
# Single env training without optuna - for simplicity
def train(args, start_model_path=None):
    run_name = f"{args.env_id}__{args.exp_name}__{args.seed}__{args.run_id}"
    prefix = ""
    
    writer = SummaryWriter(f"runs/{run_name}")
    writer.add_text(
        "hyperparameters",
        "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in vars(args).items()])),
    )

    # TRY NOT TO MODIFY: seeding
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.backends.cudnn.deterministic = args.torch_deterministic

    device_name = "cuda" if torch.cuda.is_available() and args.cuda else "cpu"
    device_name = "mps" if torch.backends.mps.is_available() and args.mps else device_name
    device = torch.device(device_name)

    print("device_name:", device_name)

    # env setup
    env = get_env(args.env_id, args.seed, args.capture_video, run_name, args.video_frequency, args.frame_stack)
    assert isinstance(env.action_space, Discrete), "only discrete action space is supported"

    q_network = QNetwork(env.action_space.n, args.frame_stack).to(device)
    if start_model_path is not None:
        state_dict = torch.load(start_model_path)
        q_network.load_state_dict(state_dict)
    
    optimizer = optim.Adam(q_network.parameters(), lr=args.learning_rate)
    target_network = QNetwork(env.action_space.n, args.frame_stack).to(device)
    target_network.load_state_dict(q_network.state_dict())

    summary(q_network, input_size=(args.frame_stack, *BOARD_SHAPE), batch_size=args.batch_size, device=device_name)

    rb = ReplayBuffer(
        args.buffer_size,
        env.observation_space,
        env.action_space,
        device,
    )
    obs = env.reset()

    # Tracks number of episodes simulated
    episode_cnt = 0
    # Tracks the number of pieces we have played
    piece_count = 0
    # Whether we explore (play random moves) or exploit (play according to the model)
    explore = True
    info = None

    iteration_time = 0.0
    env_time = 0.0

    # Track the best scoring models
    scores = deque(maxlen=args.mean_score_count)
    best_mean_score = -1.0

    if args.capture_inputs_video:
        out = cv2.VideoWriter(f'episode0.mp4', cv2.VideoWriter_fourcc(*'mp4v'), FPS, (BOARD_SHAPE[1]*SCALE_UP, BOARD_SHAPE[0]*SCALE_UP), False)

    prev_time = time.time()

    for global_step in range(args.total_timesteps):

        if global_step > 0:
            iteration_time += time.time() - iteration_start
        iteration_start = time.time()

        # Print time spent doing env.step() over last 10000 iterations
        if global_step > 0 and global_step % 10000 == 0:
            portion = int(1000 * env_time / iteration_time) / 10
            print(f"Global Step = {global_step}. Env time: {portion}%")
            iteration_time = 0.0
            env_time = 0.0

        if global_step > 0 and global_step % 100 == 0:
           curr_time = time.time()
           writer.add_scalar("charts/SPS", 100 / (curr_time - prev_time), global_step)
           prev_time = curr_time
        
        if args.capture_inputs_video and (episode_cnt % args.video_frequency == 0):
            img = np.array(obs).astype('uint8')*255
            if args.frame_stack > 1:
                img = img[-1]
            img = np.repeat(np.repeat(img, SCALE_UP, axis=0), SCALE_UP, axis=1)
            out.write(img)
        
        # If a new piece has been generated, decide wether we will explore or exploit for this piece
        if (info is not None) and (piece_count != info.get("piece_count")):
            piece_count = info.get("piece_count")
            if global_step < args.learning_starts:
                epsilon = args.start_e
            else:
                duration = args.exploration_fraction * (args.total_timesteps - args.learning_starts)
                epsilon = linear_schedule(args.start_e, args.end_e, duration, global_step - args.learning_starts)
            explore = (random.random() < epsilon)

        if explore:
            action = env.action_space.sample()
        else:
            input = np.expand_dims(np.array(obs), axis=0)
            input = torch.Tensor(input).to(device)
            q_values = q_network(input)
            action = int(torch.argmax(q_values))
        
        env_start = time.time()
        # Play a step with the given action
        next_obs, reward, done, info = env.step(action)
        env_time += time.time() - env_start

        if not done:
            # Add observation to replay buffer
            rb.add(obs, next_obs, np.array([action]), [reward], [done], [info])
            obs = next_obs
        else:
            print(f"Episode {episode_cnt} completed: {prefix}global_step={global_step},\tepisodic_return={info.get('episode')['r']:.1f},\tscore={info.get('score')}")
            writer.add_scalar("charts/episodic_return", info.get("episode")["r"], global_step)
            writer.add_scalar("charts/episodic_length", info.get("episode")["l"], global_step)
            writer.add_scalar("charts/epsilon", epsilon, global_step)
            writer.add_scalar("charts/score", info.get("score"), global_step)

            episode_cnt += 1

            scores.append(info.get("score"))
            if episode_cnt > args.mean_score_count:
                mean_score = sum(scores) / args.mean_score_count
                if mean_score > best_mean_score and global_step > args.learning_starts:
                    best_mean_score = mean_score
                    print(f"New best mean score: {mean_score}")
                    # Keep a backup of the best scoring model
                    best_model_path = f"runs/{run_name}/{args.exp_name}.best"
                    torch.save(q_network.state_dict(), best_model_path)

            if args.capture_inputs_video:
                if episode_cnt % args.video_frequency == 0:
                    out = cv2.VideoWriter(f'episode{episode_cnt}.mp4', cv2.VideoWriter_fourcc(*'mp4v'), FPS, (BOARD_SHAPE[1]*SCALE_UP, BOARD_SHAPE[0]*SCALE_UP), False)
                else:
                    out = None
            
            if episode_cnt % args.reload_env_frequency == 0:
                num_reloads = episode_cnt // args.reload_env_frequency
                del env
                env = get_env(args.env_id, args.seed, args.capture_video, run_name, args.video_frequency, args.frame_stack, num_reloads)
            
            obs = env.reset()

        # Training Logic
        if global_step > args.learning_starts:
            if global_step % args.train_frequency == 0:
                data = rb.sample(args.batch_size)
                with torch.no_grad():
                    target_max, _ = target_network(data.next_observations).max(dim=1)
                    td_target = data.rewards.flatten() + args.gamma * target_max * (1 - data.dones.flatten())
                old_val = q_network(data.observations).gather(1, data.actions).squeeze()
                # loss = F.mse_loss(td_target, old_val)
                loss = F.mse_loss(old_val, td_target)

                if global_step % 100 == 0:
                    writer.add_scalar("losses/td_loss", loss, global_step)
                    writer.add_scalar("losses/q_values", old_val.mean().item(), global_step)

                # optimize the model
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            # update target network
            if global_step % args.target_network_frequency == 0:
                for target_network_param, q_network_param in zip(target_network.parameters(), q_network.parameters()):
                    target_network_param.data.copy_(
                        args.tau * q_network_param.data + (1.0 - args.tau) * target_network_param.data
                    )

            if global_step % args.backup_frequency == 0:
                best_model_path = f"runs/{run_name}/{args.exp_name}.backup"
                torch.save(q_network.state_dict(), best_model_path)

    if args.save_model:
        model_path = f"runs/{run_name}/{args.exp_name}.cleanrl_model"
        torch.save(q_network.state_dict(), model_path)
        print(f"{prefix}model saved to {model_path}")

        scores = evaluate(
            q_network,
            args.env_id,
            args.eval_episodes,
            run_name=f"{run_name}-eval",
            seed=args.seed,
            device=device,
            capture_video=args.capture_video,
            frame_stack=args.frame_stack
        )

        print("Eval Scores:", scores)
        
    env.close()
    writer.close()

## Main

In [21]:
class Args:
    def __init__(self):
        # Settings
        self.exp_name = "Tetris_DQN"
        self.run_id = int(time.time())
        self.torch_deterministic = True
        self.cuda = True
        self.mps = False
        self.capture_video = True
        self.capture_inputs_video = True
        self.save_model = True
        self.eval_episodes = 1
        self.backup_frequency = 10000
        self.mean_score_count = 50
        self.video_frequency = 50
        self.reload_env_frequency = 49

        # Hyper-Parameters
        self.env_id = "TetrisA-v5"
        self.frame_stack = 6
        self.seed = 2
        self.total_timesteps = 500_000
        self.learning_rate = 2e-4
        self.buffer_size = 10_000
        self.gamma = 0.99
        self.tau = 0.999
        self.target_network_frequency = 1000
        self.batch_size = 32
        self.start_e = 0.1
        self.end_e = 0.1
        self.exploration_fraction = 0.2
        self.learning_starts = 10_000
        self.train_frequency = 1

args = Args()

In [12]:
# !rm -r runs/* videos/* model_input_videos/* episode*.mp4

rm: cannot remove 'model_input_videos/*': No such file or directory


In [15]:
prev_run_name = f"{args.env_id}__{args.exp_name}__{args.seed}__{1689939589}"
prev_model_path = f"runs/{prev_run_name}/{args.exp_name}.cleanrl_model"

In [22]:
train(args, start_model_path=prev_model_path)

device_name: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                 [32, 1200]               0
            Linear-2                 [32, 1024]       1,229,824
              ReLU-3                 [32, 1024]               0
            Linear-4                  [32, 512]         524,800
              ReLU-5                  [32, 512]               0
            Linear-6                    [32, 6]           3,078
Total params: 1,757,702
Trainable params: 1,757,702
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.15
Forward/backward pass size (MB): 1.04
Params size (MB): 6.71
Estimated Total Size (MB): 7.90
----------------------------------------------------------------
Episode 0 completed: global_step=220,	episodic_return=-60.3,	score=0
Episode 1 completed: global_step=446,	episodic_return=-59.5,	score=0
Episode 2 

KeyboardInterrupt: 

In [23]:
device = 'cuda'
run_name = f"{args.env_id}__{args.exp_name}__{args.seed}__{args.run_id}"
model_path = f"runs/{run_name}/{args.exp_name}.best"
state_dict = torch.load(model_path) #, device)
net = QNetwork(len(SIMPLE_MOVEMENT), args.frame_stack).to(device)
net.load_state_dict(state_dict)

<All keys matched successfully>

In [24]:
score = evaluate(net, args.env_id, 1, f'{run_name}2-eval', 2, device, True, 1, args.frame_stack)
# score = evaluate(net, args.env_id, 1, '', 20, device, True, 1, args.frame_stack, 0.1)

eval_episode=0, score=3700, episodic_return=3640.49853515625


In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs